# HW 12 - Regression

## Task 1.
For the rest of today's session let us do an example of regression analysis. Pick a dataset from the UCI repository or from sklearn. For example yo can select the Computer hardware dataset(http://archive.ics.uci.edu/ml/datasets/Computer+Hardware) and try to analyse how different parameters of computer hardware affect its perfromance. You can select any other dataset, which is tagged for regression, but be ready, that you might need to additional transformations of the initial data to perform regression.

Start with linear regression, check the results, then try to use polynomial regression and regularization if necessary. At every step analyse the results, which you are getting.

## Task 2.
Use polynomial regression to fit LS/MS time-series data. Do the following:

1. Pick any file from the "Data/LS_MS Data CSV" folder.
2. Use polynomial regression to fit time series data for each mass separetly. You have to select an approprieate degree of the regression polynomial, use regularization, if necessary.
3. Check the results of the acquired fit. Use goodness-of-fit analysis and analysis of the residuals. In this case we can assume, that residuals must follow a normal distribution, but we don't know its parameters.
4. If your analysis will show inconsistent results, then try to figure out what should be adjusted. You can change parameters of the model, correct regularization parameter or preprocess data.

## Task 3.
Use non-linear regression to fit LS/MS time-series data. When provided time-series data contains one peak and its shape resembles a Gaussian bell curve, we can try to use non-linear regression to directly fit a normal distribution probability density function to the data. Do the following:

1. Pick any file from the "Data/LS_MS Data CSV" folder, which contains one peak and its shape resembles a Gaussian bell curve,.
2. Use nonlinear regression to fit time series data for each mass separetly. You will basically regression the parameters of the distribution (e.g. distribution mean value and variance).
3. Check the results of the acquired fit. Use goodness-of-fit analysis and analysis of the residuals. Again, we can assume, that residuals must have normal distribution.